### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [ ]:
data = pd.read_csv("Data/Churn_Modelling.csv")
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis = 1)

X = data.drop("Exited", axis=1)
y = data["Exited"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

onehot_encoder_gender = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_gender_encoded = onehot_encoder_gender.fit_transform(X_train[["Gender"]])
X_test_gender_encoded = onehot_encoder_gender.transform(X_test[["Gender"]])

onehot_encoder_geo = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_geo_encoded = onehot_encoder_geo.fit_transform(X_train[["Geography"]])
X_test_geo_encoded = onehot_encoder_geo.transform(X_test[["Geography"]])

gender_encoded_df = pd.DataFrame(X_train_gender_encoded, columns=onehot_encoder_gender.get_feature_names_out(["Gender"]))
geo_encoded_df = pd.DataFrame(X_train_geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

gender_train_df = pd.DataFrame(X_train_gender_encoded, 
                           columns=onehot_encoder_gender.get_feature_names_out(["Gender"]),
                           index=X_train.index)
gender_test_df = pd.DataFrame(X_test_gender_encoded,
                          columns=onehot_encoder_gender.get_feature_names_out(["Gender"]),
                          index=X_test.index)

geo_train_df = pd.DataFrame(X_train_geo_encoded, 
                           columns=onehot_encoder_geo.get_feature_names_out(["Geography"]),
                           index=X_train.index)
geo_test_df = pd.DataFrame(X_test_geo_encoded,
                          columns=onehot_encoder_geo.get_feature_names_out(["Geography"]),
                          index=X_test.index)

X_train = pd.concat([X_train.drop(["Geography", "Gender"], axis=1), geo_train_df, gender_train_df], axis=1)
X_test = pd.concat([X_test.drop(["Geography", "Gender"], axis=1), geo_test_df, gender_test_df], axis=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
## Save the encoders and scaler (done)
with open ("artifacts/preprocessing/OHE_gender.pkl", "wb") as file:
    pickle.dump(onehot_encoder_gender, file)
with open("artifacts/preprocessing/OHE_geo.pkl", "wb") as file:
    pickle.dump(onehot_encoder_geo, file)
with open('artifacts/preprocessing/scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [3]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model


In [4]:
## Create a Keras classifier
model = KerasClassifier(
    model=create_model,
    verbose=1
)


In [5]:

# Define the grid search parameters
param_grid = {
    'model__neurons': [16, 32, 64, 128],
    'model__layers': [1, 2],
    'epochs': [50, 100]
}

In [6]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


Epoch 1/100


250/250 [==============================] - 5s 4ms/step - loss: 0.6080 - accuracy: 0.6741
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4631 - accuracy: 0.8041
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4359 - accuracy: 0.8119
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4221 - accuracy: 0.8207
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4105 - accuracy: 0.8288
Epoch 6/100
250/250 [==============================] - 3s 12ms/step - loss: 0.3980 - accuracy: 0.8341
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3862 - accuracy: 0.8388
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3768 - accuracy: 0.8406
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3692 - accuracy: 0.8444
Epoch 10/100
250/250 [===